## Getting ERA5-Land monthly via Python API

In [1]:
import cdsapi
import timeit
import os
from utils import *
import xarray as xr
import zipfile
from matplotlib import pyplot as plt

### User Input

In [2]:
ISO_A2 = 'GN'

start_year = 1950                                            # from 1950
end_year = 2025

variables_list = ['2m_temperature',
                  'total_precipitation',
                  'runoff',
                  'potential_evaporation',
                  'total_evaporation'
                 ]# to present year

folder = 'era5'
if not os.path.exists(folder): os.mkdir(folder)
folder_api = os.path.join(folder, 'era5_api')
if not os.path.exists(folder_api): os.mkdir(folder_api)
folder_extract = os.path.join(folder, 'era5_extract')
if not os.path.exists(folder_extract): os.mkdir(folder_extract)
folder_output = os.path.join(folder, 'output')
if not os.path.exists(folder_output): os.mkdir(folder_output)
folder_output = os.path.join(folder_output, ISO_A2)
if not os.path.exists(folder_output): os.mkdir(folder_output)

### Processing Data

In [3]:
dataset = 'reanalysis-era5-land-monthly-means'# 'reanalysis-era5-land'
variable_name = {
    'total_precipitation': 'tp',
    'surface_runoff': 'sro',
    'runoff': 'ro',
    'snow_depth_water_equivalent': 'sd',
    '2m_temperature': 't2m',
    'potential_evaporation': 'pev',
    'total_evaporation': 'e'
}

temp = '_'.join([variable_name[variable] for variable in variables_list])
downloaded_file = f'{dataset}_{ISO_A2}_{start_year}_{end_year}_{temp}.zip'
print(f'File to download: {downloaded_file}')

long_west, lat_south, long_east, lat_north = get_bbox(ISO_A2)
print(long_west, lat_south, long_east, lat_north)


File to download: reanalysis-era5-land-monthly-means_GN_1950_2025_t2m_tp_ro_pev_e.zip
-15.081125454999949 7.190208232000103 -7.662447469999961 12.673387757000029


### API Climate Data Store

In [4]:
years = [ str(start_year +i ) for i in range(end_year - start_year + 1)]
if not os.path.exists(folder_api): os.mkdir(folder_api)
downloaded_file = os.path.join(folder_api, downloaded_file)

if not os.path.exists(downloaded_file):
    print('Process started. Please wait the ending message ... ')
    start = timeit.default_timer()
    c = cdsapi.Client()

    c.retrieve(
        dataset,
        {
            'format': 'netcdf',
            #'format': 'grib',
            'product_type': 'monthly_averaged_reanalysis',
            'variable': variables_list,
            'year': years,
            'month': [ '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12' ],
            'time': '00:00',
            'area': [ lat_south, long_west, lat_north, long_east ],
        }, downloaded_file
        )

    stop = timeit.default_timer()
    print('Process completed in ', (stop - start)/60, ' minutes')
else:
    print('File already exists.')

File already exists.


In [5]:
ds_raw = extract_data(downloaded_file, step_type=True, extract_to=folder_extract)
print(ds_raw)

Opening GRIB file: era5/era5_extract/reanalysis-era5-land-monthly-means_GN_1950_2025_t2m_tp_ro_pev_e.grib


Ignoring index file 'era5/era5_extract/reanalysis-era5-land-monthly-means_GN_1950_2025_t2m_tp_ro_pev_e.grib.5b7b6.idx' older than GRIB file
Ignoring index file 'era5/era5_extract/reanalysis-era5-land-monthly-means_GN_1950_2025_t2m_tp_ro_pev_e.grib.5b7b6.idx' older than GRIB file
Ignoring index file 'era5/era5_extract/reanalysis-era5-land-monthly-means_GN_1950_2025_t2m_tp_ro_pev_e.grib.5b7b6.idx' older than GRIB file


Opening GRIB file: era5/era5_extract/reanalysis-era5-land-monthly-means_GN_1950_2025_t2m_tp_ro_pev_e.grib


Ignoring index file 'era5/era5_extract/reanalysis-era5-land-monthly-means_GN_1950_2025_t2m_tp_ro_pev_e.grib.5b7b6.idx' older than GRIB file


<xarray.Dataset> Size: 75MB
Dimensions:     (time: 903, latitude: 55, longitude: 75)
Coordinates:
  * time        (time) datetime64[ns] 7kB 1950-01-01 1950-02-01 ... 2025-03-01
  * latitude    (latitude) float64 440B 12.59 12.49 12.39 ... 7.391 7.291 7.19
  * longitude   (longitude) float64 600B -15.08 -14.98 -14.88 ... -7.781 -7.681
    number      int64 8B 0
    step        timedelta64[ns] 8B 1 days
    surface     float64 8B 0.0
    valid_time  (time) datetime64[ns] 7kB 1950-01-02 1950-02-02 ... 2025-03-02
Data variables:
    t2m         (time, latitude, longitude) float32 15MB ...
    tp          (time, latitude, longitude) float32 15MB 1.261e-06 ... 0.002352
    ro          (time, latitude, longitude) float32 15MB 0.0001033 ... 3.789e-05
    pev         (time, latitude, longitude) float32 15MB -0.0111 ... -0.009597
    e           (time, latitude, longitude) float32 15MB -0.002137 ... -0.002698
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_c

In [6]:
calculate_resolution_netcdf(ds_raw, lat_name='latitude', lon_name='longitude')

for var in ds_raw.data_vars:
    print(f"{var}: {ds_raw[var].attrs['units']}")

Spatial resolution: 0.10001351351351317° lon x -0.09999999999999964° lat
Approximate spatial resolution:
10.94 km (lon_name) x -11.10 km (lat_name) at 9.89° lat
Temporal resolution: 31 days
t2m: K
tp: m
ro: m
pev: m
e: m of water equivalent


### Unit Conversion

In [7]:
ds = convert_dataset_units(ds_raw)

## Export prepared data for future analysis

In [8]:
ds.to_netcdf(os.path.join(folder_output, f"{ os.path.splitext(os.path.basename(downloaded_file))[0]}.nc"))

In [9]:
plot_spatial_mean_timeseries_all_vars(ds, folder=folder_output)
plot_monthly_climatology_grid(ds, "ro", folder=folder_output)
plot_monthly_climatology_grid(ds, "tp", folder=folder_output)

In [10]:
ds['ro'].units.replace('/', '-')

'mm-day'

## Spatial Mean

In [11]:
catchment_geojson = 'catchment_files/stationbasins_guinea.geojson'
data_catchment = gpd.read_file(catchment_geojson)

df = calculate_spatial_mean_annual(ds, data_catchment, lat_name='latitude', lon_name='longitude')
df.to_csv(os.path.join(folder_output, f'{dataset}_{ISO_A2}_monthly.csv'), index=False)

df_yearly = convert_to_yearly_mm_year(df.copy(), var_name="ro", unit_init='mm/day')
df_yearly.replace(0, pd.NA, inplace=True) # Assume 0 is missing data
df_yearly.pivot(index='year', columns='region').to_csv(os.path.join(folder_output, f'{dataset}_{ISO_A2}_runoff_mm-year.csv'), index=True)

df_yearly = convert_to_yearly_mm_year(df.copy(), var_name="tp", unit_init='mm/day')
df_yearly.pivot(index='year', columns='region').to_csv(os.path.join(folder_output, f'{dataset}_{ISO_A2}_precipitation_mm-year.csv'), index=True)


📦 DataArray bounds:
  lon_name (longitude): -15.0820 to -7.6810
  lat_name (latitude):  7.1900 to 12.5910
  lon_name: -13.0750 to -7.6583
  lat_name:  7.9917 to 11.9167

✅ Overlap detected.

Region BADERA

📦 DataArray bounds:
  lon_name (longitude): -15.0820 to -7.6810
  lat_name (latitude):  7.1900 to 12.5910
  lon_name: -12.8750 to -12.2000
  lat_name:  9.8500 to 10.4542

✅ Overlap detected.

Region DIAWLA

📦 DataArray bounds:
  lon_name (longitude): -15.0820 to -7.6810
  lat_name (latitude):  7.1900 to 12.5910
  lon_name: -12.4333 to -12.2208
  lat_name:  11.0708 to 11.3375

✅ Overlap detected.

Region NONGOA

📦 DataArray bounds:
  lon_name (longitude): -15.0820 to -7.6810
  lat_name (latitude):  7.1900 to 12.5910
  lon_name: -10.3875 to -9.2667
  lat_name:  8.3042 to 9.1833

✅ Overlap detected.

Region BAC

📦 DataArray bounds:
  lon_name (longitude): -15.0820 to -7.6810
  lat_name (latitude):  7.1900 to 12.5910
  lon_name: -9.3542 to -8.6500
  lat_name:  7.9917 to 8.8167

✅ Overla